<a href="https://colab.research.google.com/github/NasrinMokhtarian/Zoomcamp-2025./blob/main/batch_processing/codes/homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pyspark, pandas as pd
from pyspark.sql import SparkSession, types, functions as F
spark = SparkSession.builder.master('local[*]').appName('module5').getOrCreate()

####Q1:spark.version
. answer: '3.5.5'

In [ ]:
spark.version

In [ ]:
df_y = spark.read.parquet('data/yellow_tripdata_2024-10.parquet')
df_y.repartition(4).write.parquet('data/homework/yellow_2024_10','overwrite')
df_y.printSchema()

root
  ###### |-- VendorID: integer (nullable = true)
  ###### |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 ###### |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 ###### |-- passenger_count: long (nullable = true)
 ###### |-- trip_distance: double (nullable = true)
 ###### |-- RatecodeID: long (nullable = true)
 ###### |-- store_and_fwd_flag: string (nullable = true)
 ###### |-- PULocationID: integer (nullable = true)
 ###### |-- DOLocationID: integer (nullable = true)
 ###### |-- payment_type: long (nullable = true)
 ###### |-- fare_amount: double (nullable = true)
 ###### |-- extra: double (nullable = true)
 ###### |-- mta_tax: double (nullable = true)
 ###### |-- tip_amount: double (nullable = true)
 ###### |-- tolls_amount: double (nullable = true)
 ###### |-- improvement_surcharge: double (nullable = true)
 ###### |-- total_amount: double (nullable = true)
 ###### |-- congestion_surcharge: double (nullable = true)
 ###### |-- Airport_fee: double (nullable = true)




####Q2:
Read the October 2024 Yellow into a Spark Dataframe.
Repartition the Dataframe to 4 partitions and save it to parquet.
What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)? Select the answer which most closely matches.

1.  6MB
2.  25MB
3.  75MB
4.  100MB



> answer is :100MB

In [ ]:
df_y.registerTempTable('trip_data')

####Q3:How many taxi trips were there on the 15th of October?

Consider only trips that started on the 15th of October.


1. 85,567
2. 105,567
3. 125,567
4. 145,567


In [ ]:

trips=spark.sql("""
select count(*) from trip_data where day(tpep_pickup_datetime)=15
""").show()

128893

####Q4:What is the length of the longest trip in the dataset in hours?

1. 122
2. 142
3. 162
4. 182

In [ ]:

longest_trip=spark.sql("""
select  MAX((UNIX_TIMESTAMP(tpep_dropoff_datetime)-UNIX_TIMESTAMP(tpep_pickup_datetime))/3600) as longest_trip
from trip_data

""").show()

162.61777777777777

####Q5:Spark’s User Interface which shows the application's dashboard runs on which local port?

1. 80
2. 443
3. 4040
4. 8080


 .  answer:4040

####Q:6Using the zone lookup data and the Yellow October 2024 data, what is the name of the LEAST frequent pickup location Zone?

1. Governor's Island/Ellis Island/Liberty Island
2. Arden Heights
3. Rikers Island
4. Jamaica Bay

  .  answer:Governor's Island/Ellis Island/Liberty Island

In [ ]:
df_zone = spark.read.option('header','true').csv('taxi_zone_lookup.csv')
df_join = df_y.join(df_zone,df_y.PULocationID==df_zone.LocationID)
df_join.show()
df_join.registerTempTable('joined')

In [ ]:
spark.sql("""
select Zone, count(tpep_pickup_datetime) as frequency
from joined
group by Zone
order by 2
""").show()